In [17]:
ClearAll["Global`*"]

In [126]:
Nval = 5; (* number of nodes *)
I0 = 1; (* initial infected *)

vars = Table[p[k][t], {k, 0, Nval}];
Print["vars:\n ", vars, "\n"]

mu = 1; (* recovery rate mu is fixed *)

(* check for various values of (beta1, beta2) pairs *)
betaPairs = {
  {0.023, 0.00042},
  {2.0 / Nval, 4.0 / Nval^2},
  {2.0 / Nval, 3.0 / Nval^2},
  {2.0 / Nval, 2.0 / Nval^2},    
  {1.5 / Nval, 3.0 / Nval^2},
  {0.01, 0.0005}  
}
Print["beta1  \t beta2 "];
Print[MatrixForm[betaPairs]];


vars:{p[0][t], p[1][t], p[2][t], p[3][t], p[4][t], p[5][t]}
 
beta1  	 beta2 
0.023     0.00042

0.4       0.16

0.4       0.12

0.4       0.08

0.3       0.12

0.01      0.0005


{{0.023, 0.00042}, {0.4, 0.16}, {0.4, 0.12}, {0.4, 0.08}, {0.3, 0.12}, {0.01, 0.0005}}

In [ ]:
(* list all N + 1 ODE equations *)
odes = Table[
   Which[ (* handle boundry states k = 0 and k = N *)
    k == 0,
    p[k]'[t] == c[k + 1] p[k + 1][t],

    k == Nval,
    p[k]'[t] == (a[k - 1] + b[k - 1]) p[k - 1][t] - c[k] p[k][t],

    True, (* interior states 1 <= k <= N - 1 *)
    p[k]'[t] == (a[k - 1] + b[k - 1]) p[k - 1][t] - (a[k] + b[k] + c[k]) p[k][t] + c[k + 1] p[k + 1][t]
    ],
   {k, 0, Nval} (* for all states k = 0, 1, ..., N *)
];
Print["KEs for N = ", Nval, ":\n", odes, "\n"];

In [16]:
pOftMatrixExp = MatrixExp[matrixQ t] . p0;